# CS-GY 9223-E: Deep Learning Homework 1
Due on Sunday, 11th February 2018, 11:55 PM

This homework can be done in pairs. Everyone must submit on NYU Classes individually.

Write down the UNIs (NetIDs) of your group (if applicable)

Member 1: Name, NetID

Member 2: Name, NetID

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [43]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0, activation="relu"):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = 3
        self.layer_dimensions = layer_dimensions
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        self.activation = activation
        self.train_cost = []
        self.val_cost = []
        self.train_accuracy = []
        self.val_accuracy = []
        self.cost=[]
        
        # init parameters        
        model = {}
        
        for i in range(len(layer_dimensions) - 1):
            layer_ahead = layer_dimensions[i+1]
            layer_behind = layer_dimensions[i]
            model['W'+str(i+1)] = np.random.randn(layer_ahead, layer_behind) * np.sqrt(2.0/(layer_ahead+layer_behind))
            model['b'+str(i+1)] = np.zeros((layer_ahead,1))
            
        self.parameters = model

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        samples = A.shape[1]
        features = A.shape[0]
        
        Z = np.dot(W,A) + b
        return Z

    def activationForward(self, A):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        if self.activation == "relu":
            A = self.relu(A)
        elif self.activation == "lrelu":
            A = self.leaky_relu(A,0.1)
        return A


    def relu(self, X):
        activation = np.maximum(0,X)
        return activation
    
    def leaky_relu(self, X, alpha):
        x = X.copy()
        out = self.relu(x) - alpha * self.relu(-x)
        return out
        
    def swish(self, X):
        x = X.copy()
        out = x * self.sigmoid(x)
        return out
        
    def sigmoid(self, X):
        x = X.copy()
        out = 1/(1 + np.exp(-x))
        return out
            
    def softmax(self, X):
        S = X - np.amax(X, axis=0)
        S = np.exp(S)
        total = np.sum(S,axis=0, keepdims=True)
        soft_m = S/total
        
        return soft_m
##    
    def dropout(self, A, prob):
        """
        :param A: Activation
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M >= prob)*1.0
        M /= (1- prob)
        A *= M
        return A, M
##

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = {}
        cache['A' + str(0)] = X
        ls = self.num_layers
        
        for i in range(ls - 2):
            W = self.parameters['W' + str(i+1)]
            b = self.parameters['b' + str(i+1)]
            A = cache['A' + str(i)]
            Z = self.affineForward(A, W, b)
            A_next = self.activationForward(Z)
##            
            # adding dropout here
            if self.drop_prob > 0 :
                A_next, M = self.dropout(A_next, self.drop_prob)
                cache['M' + str(i+1)] = M            
##        
            cache['A'+str(i+1)] = A_next
            cache['Z'+str(i+1)] = Z
        
        W = self.parameters['W'+str(ls-1)]
        b = self.parameters['b'+str(ls-1)]
        A_class = cache['A'+str(ls-2)]
        
        AL = self.affineForward(A_class, W, b) #without softmax. softmax will be applied while calculating the cost function
        
        return AL, cache
    
    def costFunction(self, y_hat, y):


        loss = (y_hat-y)**2
        s=0
        print(loss)
        for i in loss:
            s=s+i
        loss= s/2
        self.cost.append(loss)
        
        dAL = y_hat - y
        return loss, dAL


    def affineBackward(self, dA_prev, cache, layer):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        A = cache['A'+str(layer-1)]
        W = self.parameters['W'+str(layer)]
        samples = dA_prev.shape[1]
##        
        #add dropout back propagation here
        if self.drop_prob > 0:
            dA_prev = self.dropout_backward(dA_prev, cache, layer)
 ##       
        dZ = self.activationBackward(dA_prev, cache, layer, "relu")
        dW = 1/samples * np.dot(dZ, A.T)
        db = 1/samples * np.sum(dZ, axis = 1, keepdims = True)
        dA = np.dot(W.T, dZ)
        
        return dA, dW, db

    def activationBackward(self, dA, cache, layer, activation):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        Z = cache['Z'+str(layer)]
        if activation == "relu":
            dx = self.relu_derivative(dA, Z)
        elif activation == "lrelu":
            dx = self.lrelu_derivative(dA, Z, 0.1)
        return dx
    
    def relu_derivative(self, dx, cached_x):
        dA = dx * 1 * (cached_x > 0)
        return dA

    def lrelu_derivative(self, dx, cached_x, alpha):
        dA = np.ones_like(cached_x)
        dA[cached_x < 0] = alpha
        return dA*dx
##    
    def dropout_backward(self, dA, cache, layer):
        m= cache['M' + str(layer)]
        return np.multiply(m, dA) / (1 - self.drop_prob)
##
    def backPropagation(self, dAL, AL, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = {}
        
        m = AL.shape[1]
        
        samples = AL.shape[1]
        features = AL.shape[0]
        layers = self.num_layers
        
        #computing gradients using error at the output, bias at the output, and the weight matrix immediately before
        A = cache['A' + str(layers - 2)] # Activations at the layer previous layer
        W = self.parameters['W' + str(layers - 1)]
        dZ = dAL
        dW = 1/samples * np.dot(dZ, A.T)
        db = 1/samples * np.sum(dZ, axis = 1, keepdims = True)
        dA = np.dot(W.T, dZ)
        
        gradients['dW' + str(self.num_layers - 1)] = dW
        gradients['db' + str(self.num_layers - 1)] = db
        
        
        for layer in range(layers-2,0,-1):
            dA, dW, db = self.affineBackward(dA, cache, layer)
            gradients['dW' + str(layer)] = dW
            gradients['db' + str(layer)] = db
##            

            
        if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
            for i in range(self.num_layers - 2):
                dW = gradients['dW' + str(i + 1)]
                W = self.parameters['W' + str(i + 1)]
                dW_updated = dW + self.reg_lambda / m * W
                gradients['dW' + str(i + 1)] = dW_updated
##        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        for l in range(1, self.num_layers):
            self.parameters["W" + str(l)] = self.parameters["W" + str(l)] - alpha * gradients["dW" + str(l)]
            self.parameters["b" + str(l)] = self.parameters["b" + str(l)] - alpha * gradients["db" + str(l)]
        
    def train(self, X, y, iters=1000, alpha=0.001, batch_size=100, print_every=100):
        
        
        for i in range(0, iters):
            X_batch= X
            y_batch=y
            
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)
            
            # compute loss
            cost, dAL = self.costFunction(AL, y_batch)

            # compute gradients
            gradients = self.backPropagation(dAL, AL, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
    


            if i % print_every == 0:
                
                if i % 500 == 0 and i != 0:
                    alpha /= 2


                print("prediction on the training set" )
                v_pred = self.predict(X_batch)
                print(v_pred)
                for i in range(len(v_pred)):
                    if v_pred[i]>=0.5:
                        v_pred[i]=1
                    else:
                        v_pred[i]=0
                print("% inaccuracy")
                c=0  
                for i in range(len(v_pred)):
                    if v_pred[i]!=y_batch[i]:
                        c= c+1
                print(float(c)/len(v_pred)*100)

                
                
    def predict(self, X):
        """
        Make predictions for each sample
        """
        AL, _ = self.forwardPropagation(X)
        y_pred = np.argmax(AL, axis=0)
        
        return y_pred


In [44]:
# Load the data


## Part 1

#### Simple fully-connected deep neural network

In [45]:
layer_dimensions = [2,2,1]  # including the input and output layers
NN = NeuralNetwork(layer_dimensions, 0.2, 0.1,'relu')
y_train=np.array([0,1,1,0])
X_train= np.array([[0,0,1,1],[0,1,0,1]])
NN.train(X_train, y_train, iters=20000, alpha=0.3, print_every=1000)

[[0.00000000e+00 1.00000000e+00 2.10008025e-04 3.99950808e-01]]
prediction on the training set
[0 0 0 0]
% inaccuracy
50.0
[[0.         1.         1.         0.39397404]]
[[0.00000000e+00 1.00000000e+00 4.47100854e-07 3.88086593e-01]]
[[0.         1.         1.         0.38228712]]
[[0.00000000e+00 1.00000000e+00 2.43435316e-04 3.76574321e-01]]
[[0.         1.         1.         0.37094689]]
[[0.00000000e+00 1.00000000e+00 9.18878285e-04 3.65403551e-01]]
[[0.         1.         0.00141268 0.35994305]]
[[0.         1.         0.00200738 0.35456415]]
[[0.         1.         0.00270065 0.34926563]]
[[0.         1.         0.00349021 0.3440463 ]]
[[0.         1.         0.00437382 0.33890495]]
[[0.         1.         0.0053493  0.33384044]]
[[0.         1.         0.00641446 0.32885162]]
[[0.        1.        0.0075672 0.       ]]
[[0.         1.         0.00880543 0.        ]]
[[0.         1.         0.01012709 0.        ]]
[[0.         1.         1.         0.30963076]]
[[0. 1. 1. 0.]]
[

[[0.0000000e+00 1.0000000e+00 9.4275616e-01 3.2780317e-04]]
[[0.00000000e+00 1.00000000e+00 9.43179209e-01 3.22904561e-04]]
[[0.00000000e+00 1.00000000e+00 9.43599180e-01 3.18079156e-04]]
[[0.00000000e+00 1.00000000e+00 9.44016093e-01 3.13325861e-04]]
[[0.00000000e+00 1.00000000e+00 9.44429971e-01 3.08643597e-04]]
[[0.00000000e+00 1.00000000e+00 9.44840834e-01 3.04031305e-04]]
[[0.00000000e+00 1.00000000e+00 9.45248704e-01 2.99487937e-04]]
[[0.00000000e+00 1.00000000e+00 9.45653602e-01 2.95012464e-04]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 2.90603871e-04]]
[[0.0000000e+00 1.0000000e+00 1.0000000e+00 2.8626116e-04]]
[[0.00000000e+00 1.00000000e+00 9.46850673e-01 2.81983345e-04]]
[[0.00000000e+00 1.00000000e+00 9.47243892e-01 2.77769456e-04]]
[[0.00000000e+00 1.00000000e+00 9.47634242e-01 2.73618539e-04]]
[[0.00000000e+00 1.00000000e+00 9.48021744e-01 2.69529652e-04]]
[[0.00000000e+00 1.00000000e+00 9.48406418e-01 2.65501868e-04]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 2

[[0.00000000e+00 1.00000000e+00 9.97702671e-01 5.13328518e-07]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 5.05657465e-07]]
[[0.00000000e+00 1.00000000e+00 9.97736982e-01 4.98101046e-07]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 4.90657549e-07]]
[[0.00000000e+00 1.00000000e+00 9.97770781e-01 4.83325285e-07]]
[[0.00000000e+00 1.00000000e+00 9.97787491e-01 4.76102593e-07]]
[[0.00000000e+00 1.00000000e+00 9.97804076e-01 4.68987835e-07]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 4.61979398e-07]]
[[0.00000000e+00 1.00000000e+00 9.97836873e-01 4.55075693e-07]]
[[0.00000000e+00 1.00000000e+00 9.97853088e-01 4.48275156e-07]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 4.41576244e-07]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 4.34977439e-07]]
[[0.00000000e+00 1.00000000e+00 9.97901007e-01 4.28477245e-07]]
[[0.         1.         0.99791674 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.97932358e-01 4.15766817e-07]]
[[0.00000000e+00 1.00000000e+00 9.97947857e-01 4.0955370

[[0.00000000e+00 1.00000000e+00 9.99760006e-01 5.59632170e-09]]
[[0.00000000e+00 1.00000000e+00 9.99760906e-01 5.55442798e-09]]
[[0.00000000e+00 1.00000000e+00 9.99761802e-01 5.51284788e-09]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 5.47157905e-09]]
[[0.00000000e+00 1.00000000e+00 9.99763585e-01 5.43061915e-09]]
[[0.         1.         0.99976447 0.        ]]
[[0.         1.         0.99976536 0.        ]]
[[0.         1.         0.99976624 0.        ]]
[[0.         1.         0.99976711 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99767985e-01 5.23037335e-09]]
[[0.0000000e+00 1.0000000e+00 1.0000000e+00 5.1912191e-09]]
[[0.         1.         0.99976972 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99770585e-01 5.11378773e-09]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 5.07550624e-09]]
[[0.00000000e+00 1.00000000e+00 9.99772302e-01 5.03751132e-09]]
[[0.         1.         0.99977316 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99774007e-01 4.96237262e-09]]
[[0.00000000

[[0.00000000e+00 1.00000000e+00 9.99961632e-01 1.43020091e-10]]
[[0.00000000e+00 1.00000000e+00 9.99961776e-01 1.41949451e-10]]
[[0.00000000e+00 1.00000000e+00 9.99961919e-01 1.40886827e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962062e-01 1.39832157e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962204e-01 1.38785382e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962346e-01 1.37746443e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962487e-01 1.36715282e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962628e-01 1.35691840e-10]]
[[0.00000000e+00 1.00000000e+00 9.99962768e-01 1.34676059e-10]]
[[0. 1. 1. 0.]]
[[0.00000000e+00 1.00000000e+00 9.99963047e-01 1.32667253e-10]]
[[0.00000000e+00 1.00000000e+00 9.99963185e-01 1.31674115e-10]]
[[0.00000000e+00 1.00000000e+00 9.99963323e-01 1.30688410e-10]]
[[0.00000000e+00 1.00000000e+00 9.99963461e-01 1.29710085e-10]]
[[0.00000000e+00 1.00000000e+00 9.99963598e-01 1.28739084e-10]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.27775351e-10]]
[[0.00000000e+00 1.00000

...

[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41958080e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41958041e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41958002e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957963e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957924e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957885e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957845e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957806e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41957767e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957728e-14]]
[[0. 1. 1. 0.]]
[[0. 1. 1. 0.]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41957571e-14]]
[[0. 1. 1. 0.]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957493e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957454e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41957415e-14]]
[[0.    

[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946751e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946732e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946712e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946692e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41946673e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946653e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946634e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946595e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946575e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41946555e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946536e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946516e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946497e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41946477e-14]]
[[0.0000

[[0.         1.         0.99999941 0.        ]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937594e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937575e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937555e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41937516e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41937496e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937438e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937418e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937399e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937379e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41937359e-14]]
[[0.0000000e+00 1.0000000e+00 1.0000000e+00 3.4193734e-14]]
[[0.0000000e+00 1.0000000e+00 1.0000000e+00 3.4193732e-14]]


[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932468e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932458e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932448e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932439e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932429e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932419e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932409e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932399e-14]]
[[0.0000000e+00 1.0000000e+00 1.0000000e+00 3.4193239e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932370e-14]]
[[0. 1. 1. 0.]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41932351e-14]]
[[0. 1. 1. 0.]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932331e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932321e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41932311e-14]]
[[0.00000000e+00 1.00000000e

[[0. 1. 1. 0.]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928203e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928183e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928173e-14]]
[[0.00000000e+00 1.00000000e+00 1.00000000e+00 3.41928164e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928154e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928144e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928134e-14]]
[[0.         1.         0.99999941 0.        ]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928115e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928105e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928095e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928085e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928076e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.41928066e-14]]
[[0.00000000e+00 1.00000000e+00 9.99999407e-01 3.4192805

In [39]:
NN.cost

[array([0.00000000e+00, 5.00000000e-01, 1.05004013e-04, 1.99975404e-01]),
 array([0.        , 0.5       , 0.5       , 0.19698702]),
 array([0.00000000e+00, 5.00000000e-01, 2.23550427e-07, 1.94043297e-01]),
 array([0.        , 0.5       , 0.5       , 0.19114356]),
 array([0.00000000e+00, 5.00000000e-01, 1.21717658e-04, 1.88287160e-01]),
 array([0.        , 0.5       , 0.5       , 0.18547344]),
 array([0.00000000e+00, 5.00000000e-01, 4.59439142e-04, 1.82701775e-01]),
 array([0.        , 0.5       , 0.00070634, 0.17997153]),
 array([0.        , 0.5       , 0.00100369, 0.17728208]),
 array([0.        , 0.5       , 0.00135032, 0.17463282]),
 array([0.        , 0.5       , 0.0017451 , 0.17202315]),
 array([0.        , 0.5       , 0.00218691, 0.16945248]),
 array([0.        , 0.5       , 0.00267465, 0.16692022]),
 array([0.        , 0.5       , 0.00320723, 0.16442581]),
 array([0.       , 0.5      , 0.0037836, 0.       ]),
 array([0.        , 0.5       , 0.00440271, 0.        ]),
 array([0.  

In [24]:
y_predicted = NN.predict(X_test)
print(y_predicted)
#save_predictions('ans1-uni', y_predicted)

NameError: name 'X_test' is not defined

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Regularizing the neural network
#### Add dropout and L2 regularization

In [29]:
sum(np.array([1,2,3]))

6

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')


    def train(self, X, y, iters=1000, alpha=0.001, batch_size=100, print_every=100):
        
        
        for i in range(0, iters):
            X_batch= X
            y_batch=y
            
            # forward prop
            AL, cache = self.forwardPropagation(X_batch)
            
            # compute loss
            cost, dAL = self.costFunction(AL, y_batch)

            # compute gradients
            gradients = self.backPropagation(dAL, AL, cache)

            # update weights and biases based on gradient
            self.updateParameters(gradients, alpha)
    
            if i % print_every == 0:
                
                if i % 500 == 0 and i != 0:
                    alpha /= 2


                print("prediction on the training set" )
                v_pred = self.predict(X_batch)
                print(v_pred)
                for i in range(len(v_pred)):
                    if v_pred[i]>=0.5:
                        v_pred[i]=1
                    else:
                        v_pred[i]=0
                print("% inaccuracy")
                c=0  
                for i in range(len(v_pred)):
                    if v_pred[i]!=y_batch[i]:
                        c= c+1
                        print(c)
                print(c/len(v_pred)*100)